# Ovary Cancer

### Import necesaary libraries

In [1]:
import os
import warnings

# Suppress all warnings globally (including future, deprecation, runtime, sklearn, etc.)
os.environ['PYTHONWARNINGS'] = 'ignore'
warnings.filterwarnings('ignore')

import time  # For timing operations
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation and analysis
import matplotlib.pyplot as plt  # For data visualization
from scipy import stats
from scipy.optimize import minimize  # For optimization tasks
from joblib import Parallel, delayed  # For parallel processing
from sklearn.linear_model import LogisticRegression, Lasso  # For logistic regression and Lasso regression
from sklearn.metrics import auc, roc_curve, roc_auc_score, log_loss, accuracy_score, recall_score, confusion_matrix, f1_score, precision_score  # For classification metrics
from sklearn.model_selection import train_test_split, KFold  # For splitting datasets and cross-validation
from sklearn.preprocessing import StandardScaler  # For feature scaling
from sklearn.svm import SVR  # For Support Vector Regression
from sklearn.ensemble import RandomForestRegressor  # For Random Forest regression
from sklearn.utils.class_weight import compute_class_weight  # For handling class imbalance
import seaborn as sns  # For advanced data visualization

# Set a random state for reproducibility
randomstate = 1729


### Data

In [2]:
# Load the Ovary Cancer dataset
df = pd.read_csv("final.data.csv")

In [3]:

# Extract features (X) and target (y) from dataset
X = df.drop(columns=['ovar_cancer'])
y = df['ovar_cancer']

# Convert to float and ensure types are correct
X = X.astype(float)
y = y.astype(int)



### Necessary functions

#### Feature selection

In [4]:
'''
Choosing features that have the highest sensitivity for pre-specified specificity.

Input:
- X: Pandas DataFrame of features.
- y: Pandas Series of classes (class column).
- SP: The value for specificity.
- lower_bound: Optional boundaries for the coefficients of the feature (should be of dimension X.shape[1] + 1).
- num_features_to_keep: The number of features to keep that maximize sensitivity at the given specificity.

Output:
- List of features (with the length of num_features_to_keep).
'''

def feature_selection(X, y, SP, lower_bound=None, num_features_to_keep=2):
    Features = list(X.columns)  # List of all feature names
    One_Top = []  # List to hold selected features
    
    def sensitivity_score(feature):
        # Calculate sensitivity for a given feature
        if len(One_Top) == 0:
            return SMAGS(X=X.loc[:, [feature]], y=y, SP=SP, lower_bound=lower_bound)['Sensitivity']
        else:
            return SMAGS(X=X.loc[:, np.append(feature, One_Top)], y=y, SP=SP, lower_bound=lower_bound)['Sensitivity']

    while len(One_Top) < num_features_to_keep:
        # Calculate sensitivity scores for all features in parallel
        results = Parallel(n_jobs=-2)(delayed(sensitivity_score)(feature) for feature in Features)
        best_feature = Features[np.argmax(results)]  # Get the feature with the highest sensitivity

        Features.remove(best_feature)  # Remove the best feature from consideration
        One_Top.append(best_feature)  # Add the best feature to the selected list

    return One_Top  # Return the list of selected features

**Modifying `feature_selection` function**

In [ ]:
# Function to safely convert complex numbers to real
def safe_real(x):
    return np.real(x) if np.iscomplexobj(x) else x

# Modify the SMAGS function to handle complex numbers
def complex_safe_smags(*args, **kwargs):
    result = SMAGS(*args, **kwargs)
    return pd.Series({k: safe_real(v) for k, v in result.items()})

# Update feature_selection to use the complex-safe SMAGS
def complex_safe_feature_selection(X, y, SP, lower_bound=None, num_features_to_keep=2):
    Features = list(X.columns)
    One_Top = []
    
    def sensitivity_score(feature):
        if len(One_Top) == 0:
            return safe_real(complex_safe_smags(X=X.loc[:, [feature]], y=y, SP=SP, lower_bound=lower_bound)['Sensitivity'])
        else:
            return safe_real(complex_safe_smags(X=X.loc[:, np.append(feature, One_Top)], y=y, SP=SP, lower_bound=lower_bound)['Sensitivity'])
    
    while len(One_Top) < num_features_to_keep:
        results = [sensitivity_score(feature) for feature in Features]
        best_feature = Features[np.argmax(results)]
        Features.remove(best_feature)
        One_Top.append(best_feature)
    
    return One_Top



##### SMAGS

In [5]:
'''
Sensitivity Maximization At Given Specificity (SMAGS):

Choosing features that have the highest sensitivity for pre-specified specificity.

Input:
- X: Features as a Pandas DataFrame.
- y: Labels as a Pandas Series of classes (0 and 1).
- SP: Threshold value for specificity (default is 0.985).
- lower_bound: Optional boundaries for the coefficients (should be of dimension X.shape[1] + 1).

Output:
A Series with coefficients of the logistic regression that maximizes sensitivity,
including intercept, threshold, specificity, sensitivity, optimization method, tolerance, jacobian approach, and AIC.
'''

def SMAGS(X, y, SP=0.985, lower_bound=None, rand=randomstate):
    n_features = X.shape[1]
    log = LogisticRegression(random_state=rand).fit(X, y)  # Fit logistic regression model
    all_coefs = np.append(log.coef_[0], log.intercept_)  # Get coefficients and intercept
    X1 = X.assign(I=1)  # Add intercept term to feature set

    def sig(M):
        # Sigmoid function
        return 1 / (1 + np.exp(-M))

    def custom_loss(coefs):
        # Custom loss function to maximize sensitivity
        m = np.matmul(X1, coefs)
        z = sig(m.astype(float))
        if len(((1 - y) * z)[(1 - y) * z != 0]) != 0:
            threshold = np.quantile(((1 - y) * z)[(1 - y) * z != 0], SP)
        else:
            threshold = 0
        y_hat = 1 * (z > threshold)  # Predicted labels based on threshold
        loss1 = np.dot(y_hat, y) / sum(y)  # Sensitivity
        return -loss1  # Return negative sensitivity for minimization

    # Set bounds for coefficients
    bnds = [(lower_bound, None) for _ in range(n_features)]
    bnds.append((None, None))  # Intercept bound

    # Define optimization methods, tolerances, and jacobian approaches
    opt_methods = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP', 'trust-constr']
    tolerance = [1, 5e-1, 1e-1, 5e-2, 1e-2, 5e-3, 1e-3, 1e-4, 1e-5, 1e-6]
    jacob_approach = ['2-point', '3-point', 'cs']

    def process_params(met, tol, jac):
        # Process parameters for optimization
        result = minimize(fun=custom_loss, x0=np.abs(all_coefs), method=met, tol=tol, bounds=bnds, jac=jac)
        a = np.matmul(X1, result.x)  # Predicted values
        b = sig(a.astype(float))
        
        if len(((1 - y) * b)[(1 - y) * b != 0]) != 0:
            c = np.quantile(((1 - y) * b)[(1 - y) * b != 0], SP)
        else:
            c = 0
        d = 1 * (b > c)  # Predicted labels based on updated threshold

        # Create a Series to hold results
        row = pd.Series(index=np.append(np.array(['B' + str(i) for i in range(1, n_features + 1)]),
                                         np.array(['B0', 'method', 'tolerance', 'jacobian', 'threshold',
                                                   'Specificity', 'Sensitivity', 'AIC'])))
        row[0:(n_features + 1)] = result.x  # Coefficients and intercept
        row['method'] = met
        row['tolerance'] = tol
        row['jacobian'] = jac
        row['threshold'] = SP
        row['Specificity'] = np.dot(1 - d, 1 - y) / sum(1 - y)  # Calculate specificity
        row['Sensitivity'] = np.dot(d, y) / sum(y)  # Calculate sensitivity
        row['AIC'] = 2 * ((n_features + 1) - log_loss(y, sig(np.matmul(X1, result.x).astype(float))))  # Calculate AIC

        return row

    # Run the optimization in parallel for different methods, tolerances, and Jacobian approaches
    results = Parallel(n_jobs=-1)(delayed(process_params)(met, tol, jac) for met in opt_methods
                                   for tol in tolerance for jac in jacob_approach)

    max_sn = pd.DataFrame(results)
    # Filter for the maximum sensitivity and then minimize AIC
    A = max_sn[max_sn.Sensitivity == max_sn.Sensitivity.max()]
    return A[A.AIC == A.AIC.min()].iloc[0]  # Return the optimal result


### Metrics

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

accs, sns, sps, aucs, sens_at_sp985s = [], [], [], [], []
selected_features_per_fold = []
n_selecting_features = 7
SP = 0.985
n_fold = 5

kfold = KFold(n_splits=n_fold, shuffle=True, random_state=randomstate)

for fold_idx, (train, test) in enumerate(kfold.split(X), 1):
    # 1. Feature selection on training data only
    fold_selected_features = complex_safe_feature_selection(
        X.loc[X.index[train]], y.loc[y.index[train]],
        SP=SP, num_features_to_keep=n_selecting_features
    )
    selected_features_per_fold.append(fold_selected_features)

    # 2. Fit SMAGS on training data with selected features
    A = SMAGS(
        X.loc[X.index[train], fold_selected_features],
        y.loc[y.index[train]], SP
    )

    # 3. Predict on test data with selected features
    X_test = X.loc[X.index[test], fold_selected_features].assign(I=1)
    coefs = np.array(A[:len(fold_selected_features) + 1], dtype=float)
    probs = sigmoid(np.matmul(X_test.values.astype(float), coefs))
    threshold = A['threshold'] if 'threshold' in A else 0.5
    y_pred = (probs > threshold).astype(int)
    y_true = y.loc[y.index[test]]

    # 4. Calculate standard metrics
    try:
        acc = accuracy_score(y_true, y_pred)
        sn = recall_score(y_true, y_pred, zero_division=0)
        cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()
        sp = tn / (tn + fp) if (tn + fp) > 0 else np.nan
        auc_ = roc_auc_score(y_true, probs)
    except Exception as e:
        print(f"Metric calculation error on Fold {fold_idx}: {e}")
        acc = sn = sp = auc_ = np.nan

    # 5. Sensitivity at 98.5% specificity (main result!)
    try:
        fpr_full, tpr_full, thresholds = roc_curve(y_true, probs)
        idx = np.where(1 - fpr_full >= SP)[0]
        if len(idx) > 0:
            sens_at_sp = tpr_full[idx[-1]]  # highest sensitivity where specificity >= SP
        else:
            sens_at_sp = np.nan
    except Exception as e:
        print(f"ROC curve error on Fold {fold_idx}: {e}")
        sens_at_sp = np.nan

    # 6. Append metrics for this fold
    accs.append(acc)
    sns.append(sn)
    sps.append(sp)
    aucs.append(auc_)
    sens_at_sp985s.append(sens_at_sp)

    # 7. Print results for this fold
    print(f"Fold {fold_idx}: Features={fold_selected_features}")
    print(f"Fold {fold_idx}: Accuracy={acc:.2f}, Sensitivity={sn:.2f}, Specificity={sp:.2f}, "
          f"AUC={auc_:.2f}, Sens@SP=98.5%={sens_at_sp:.3f}")

# Print summary statistics
print(f"\nMean Accuracy: {np.nanmean(accs):.2f} ± {np.nanstd(accs):.2f}")
print(f"Mean Sensitivity: {np.nanmean(sns):.2f} ± {np.nanstd(sns):.2f}")
print(f"Mean Specificity: {np.nanmean(sps):.2f} ± {np.nanstd(sps):.2f}")
print(f"Mean AUC: {np.nanmean(aucs):.2f} ± {np.nanstd(aucs):.2f}")
print(f"Mean Sensitivity @ 98.5% Specificity: {np.nanmean(sens_at_sp985s):.3f} ± {np.nanstd(sens_at_sp985s):.3f}")

print("\nSelected features per fold:")
for i, feats in enumerate(selected_features_per_fold, 1):
    print(f"Fold {i}: {feats}")


Fold 1: Features=['lgca125', 'lgsil_4r_pg_ml', 'lgstnfri_pg_ml', 'lgegf_pg_ml', 'lgil_1ra_pg_ml', 'lgmip_1b_pg_ml', 'lgmcp_1_pg_ml']
Fold 1: Accuracy=0.28, Sensitivity=1.00, Specificity=0.00, AUC=0.71, Sens@SP=98.5%=0.091
Fold 2: Features=['lgca125', 'lgeotaxin_pg_ml', 'lgmdc_pg_ml', 'lgil_8_pg_ml', 'lgegf_pg_ml', 'lgfgf_2_pg_ml', 'lgmip_1b_pg_ml']
Fold 2: Accuracy=0.80, Sensitivity=0.04, Specificity=1.00, AUC=0.74, Sens@SP=98.5%=0.167
Fold 3: Features=['lgca125', 'lgeotaxin_pg_ml', 'lgg_csf_pg_ml', 'lgmip_1b_pg_ml', 'lgfgf_2_pg_ml', 'lgil_1ra_pg_ml', 'lgmdc_pg_ml']
Fold 3: Accuracy=0.66, Sensitivity=0.55, Specificity=0.70, AUC=0.65, Sens@SP=98.5%=0.172
Fold 4: Features=['lgca125', 'lgeotaxin_pg_ml', 'lgegf_pg_ml', 'lgil_1ra_pg_ml', 'lgstnfri_pg_ml', 'lgip_10_pg_ml', 'lgg_csf_pg_ml']
Fold 4: Accuracy=0.78, Sensitivity=0.13, Specificity=1.00, AUC=0.67, Sens@SP=98.5%=0.200
Fold 5: Features=['lgca125', 'lgeotaxin_pg_ml', 'lgmip_1b_pg_ml', 'lgcrp_pg_ml', 'lgmcp_1_pg_ml', 'lgtnf_b_pg_ml', '